In [1]:
import os
from dotenv import load_dotenv
import pandas as pd
import json
from datetime import datetime
from markdown import markdown
from IPython.display import Markdown, display, HTML
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
load_dotenv()
import eyecite

CAP_API_KEY = os.getenv('CAP_API_KEY')

In [2]:
os.chdir(os.path.dirname(os.getcwd()))

In [3]:
from src.types import DocMetaData, Document
from src.utils.gen_utils import count_tokens, hash_id
from src.utils.pydantic_utils import dataframe_to_documents, documents_to_dataframe
from src.parsing.parser import ParsingConfig, Parser
from src.parsing.search import eliminate_near_duplicates_df, eliminate_near_duplicates_grouped
from src.citations import (
    get_citation_context_sents, 
    get_authorities_list, 
    get_top_citations,
    extract_resolved_citations,
    get_citation_context,
    create_citation_lookup_table,
)
from src.embedding_models.models import ColbertReranker
from src.utils.output import write_md_to_pdf
from justopinion.download import CAPClient

In [140]:
client = CAPClient(CAP_API_KEY)
response = client.fetch_cite(cite="104 S. Ct. 774", full_case=True)
case = json.loads(response.content)

In [152]:
len(case["results"][0]["cites_to"])

123

In [23]:
df = pd.read_parquet("data/reddit_legal_cluster_test_results.parquet")
print(f"df shape: {df.shape}")
df.head()


df shape: (5000, 14)


,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,State,kmeans_label,topic_title
1078,1078,1575952538,https://www.reddit.com/r/legaladvice/comments/...,e8lsen,I applied for a job and after two interviews I...,"Failed a drug test due to amphetamines, I have...",employment,5,"[9.475638042064453e-05, 0.0005111666301983955,...",493,"""Validity of Schedule II Drug Prescription in ...",PR,8,Employment Legal Concerns and Issues
2098,2098,1577442453,https://www.reddit.com/r/legaladvice/comments/...,eg9ll2,"Hi everyone, thanks in advance for any guidanc...","Speeding ticket in Tennessee, Georgia Driver's...",driving,4,"[-0.006706413111028856, 0.020911016696181495, ...",252,"""Speeding ticket consequences for out-of-state...",KY,10,Legal Topics in Traffic Violations
9384,9384,1412266873,https://www.reddit.com/r/legaladvice/comments/...,2i3or9,So the line in question is:\n\n*Husband shall ...,Can't figure out what this language from Divor...,family,6,"[-0.0008707255965511636, 0.015741086521669008,...",157,Interpreting divorce decree language regarding...,IL,7,Child Custody and Related Issues
5425,5425,1578732752,https://www.reddit.com/r/legaladvice/comments/...,en55d8,"I rented an apartment as a student, and I was ...",FICO score dropped 100points,housing,7,"[0.0018182859833585185, -0.021543734695078785,...",78,"""Responsibility for missed rent payments and r...",FM,4,Legal Issues in Consumer Protection
7735,7735,1578331074,https://www.reddit.com/r/legaladvice/comments/...,ekx2ch,I have a audio recording of me saying I can’t ...,[MI] I told my boss I had epilepsy and was fir...,employment,5,"[-0.004012766701303168, 0.01652626296610702, 0...",251,"""Wrongful Termination Due to Epilepsy: Seeking...",DC,8,Employment Legal Concerns and Issues


In [5]:
top_citations = get_top_citations(df, "Complete Text", 20)

In [6]:
top_citations.head(10)


,Citation,Count,link
0,464 U.S. 417,47,https://cite.case.law/us/464/417
1,104 S. Ct. 774,44,https://www.courtlistener.com/c/S.%20Ct./104/774
2,78 L. Ed. 2d 574,44,https://www.courtlistener.com/c/L.%20Ed.%202d/...
3,17 U.S.C. § 101,33,No link found
4,471 U.S. 539,29,https://cite.case.law/us/471/539
5,17 U.S.C. § 102,26,No link found
6,17 U.S.C. § 106,26,No link found
7,105 S. Ct. 2218,26,https://www.courtlistener.com/c/S.%20Ct./105/2218
8,85 L. Ed. 2d 588,26,https://www.courtlistener.com/c/L.%20Ed.%202d/...
9,17 U.S.C. § 501,23,No link found


In [7]:
top_5 = top_citations.head(5)["Citation"].tolist()
top_5
    

['464 U.S. 417',
 '104 S. Ct. 774',
 '78 L. Ed. 2d 574',
 '17 U.S.C. § 101',
 '471 U.S. 539']

In [8]:
context = get_citation_context(
    df=df, 
    id_column="Citing Case ID", 
    text_column="Complete Text", 
    sentences_before=8, 
    sentences_after=4, 
    target_citations=top_5
)

In [9]:
context

,id,citation,context,start_char,end_char
0,7399454,464 U.S. 417,"Further, the Court of Appeals stated, a plaint...",6135,7485
1,7399454,78 L. Ed. 2d 574,"[that] the copying, if proven, went so far as ...",6295,7684
2,7399454,464 U.S. 417,"Further, the Court of Appeals stated, a plaint...",41613,42963
3,7399454,78 L. Ed. 2d 574,"[that] the copying, if proven, went so far as ...",41773,43162
4,7399454,464 U.S. 417,"Nimmer on Copyright see, e.g., Atari, Inc. v....",76685,77933
...,...,...,...,...,...
1400,633900,464 U.S. 417,Brief for Columbia Pictures et al. as Amici C...,195116,196697
1401,633900,464 U.S. 417,"In any event, the complaint that respondent’s ...",195261,197275
1402,633900,17 U.S.C. § 101,The record supports the Court of Appeals’ conc...,217052,218140
1403,633900,17 U.S.C. § 101,“This case presents a classic example of an un...,217284,218478


In [10]:
context.drop_duplicates(inplace=True)
context

,id,citation,context,start_char,end_char
0,7399454,464 U.S. 417,"Further, the Court of Appeals stated, a plaint...",6135,7485
1,7399454,78 L. Ed. 2d 574,"[that] the copying, if proven, went so far as ...",6295,7684
2,7399454,464 U.S. 417,"Further, the Court of Appeals stated, a plaint...",41613,42963
3,7399454,78 L. Ed. 2d 574,"[that] the copying, if proven, went so far as ...",41773,43162
4,7399454,464 U.S. 417,"Nimmer on Copyright see, e.g., Atari, Inc. v....",76685,77933
...,...,...,...,...,...
1374,633900,464 U.S. 417,Brief for Columbia Pictures et al. as Amici C...,195116,196697
1375,633900,464 U.S. 417,"In any event, the complaint that respondent’s ...",195261,197275
1376,633900,17 U.S.C. § 101,The record supports the Court of Appeals’ conc...,217052,218140
1377,633900,17 U.S.C. § 101,“This case presents a classic example of an un...,217284,218478


In [11]:
context["citation"].value_counts()

citation
464 U.S. 417        60
78 L. Ed. 2d 574    52
17 U.S.C. § 101     34
471 U.S. 539        28
104 S. Ct. 774      13
Name: count, dtype: int64

In [12]:
distinct_context = eliminate_near_duplicates_grouped(context, "context", group_columns=["id", "citation"], threshold=0.75)
concatenated_context = distinct_context.sort_values(by=['start_char']).groupby(['citation', 'id'])['context'].apply(' '.join).reset_index()

In [13]:
print(f"shape: {concatenated_context.shape}")
concatenated_context.head()

shape: (32, 3)


,citation,id,context
0,104 S. Ct. 774,1451062,See Wainwright Sec. v. Wall Street Transcript...
1,104 S. Ct. 774,3337641,"The Claims Court’s predecessor, under the juri..."
2,104 S. Ct. 774,7396224,The court held that the commercial nature of t...
3,17 U.S.C. § 101,633900,"3 Nimmer § 13.05[A], p. 13-81. The record supp..."
4,17 U.S.C. § 101,1451062,"On the same day, the district court granted an..."


In [14]:
text = concatenated_context.head(1)["context"].tolist()[0]

In [15]:
html_text = markdown(text)

In [16]:
report = await write_md_to_pdf(html_text)

Report written to ./outputs/20240218-d5fe8.pdf


In [18]:
report

'./outputs/20240218-d5fe8.pdf'

In [51]:
from typing import List, Literal, Optional
from pydantic import BaseModel, Field

class LegalCitation(BaseModel):
    """Information about a legal ruling and its application."""
    
    citation: str = Field(
        ...,
        description="The Citation specified by the user.",
    )
    question_summary: str = Field(
        ...,
        description="A concise summary of the underlying legal question.",
    )
    rule: str = Field(
        ...,
        description="A rule, argument, decision, or otherwise legal principal the citation is begin associated with.",
    )
    application: Optional[str] = Field(
        None,
        description="Application, or potential application of the rule to the question."
    )

In [52]:
from tenacity import AsyncRetrying, stop_after_attempt, wait_fixed

In [53]:
import openai
import instructor

client = instructor.patch(openai.AsyncOpenAI())

In [60]:
async def analyze_citations(citation, context):
    return await client.chat.completions.create(
        model="gpt-4-turbo-preview",
        response_model=LegalCitation,
        max_retries=AsyncRetrying(
        stop=stop_after_attempt(3),
        wait=wait_fixed(1),
    ),
        messages=[
            {
                "role": "system",
                "content": "Your role is to extract information about a legal citation using the context provided and without prior knowledge.",
            },
            {"role": "user", "content": f"Your task focuses on citation: **{citation}**"},
            {"role": "user", "content": f"Here is the context: {context}"}
        ]
    )

In [57]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [61]:
from tqdm.asyncio import tqdm

async def process_citations_with_progress(concatenated_context):
    results = []
    for row in tqdm(concatenated_context.itertuples(), total=len(concatenated_context)):
        result = await analyze_citations(row.citation, row.context)
        results.append(result)
    return results

all_data = await process_citations_with_progress(concatenated_context)

100%|██████████| 39/39 [04:05<00:00,  6.30s/it]


In [72]:
data_dicts = [model.model_dump() for model in all_data]
df_res = pd.DataFrame(data_dicts)
df_res.head()

,citation,question_summary,rule,application
0,104 S. Ct. 774,Understanding the legal principle established ...,The case addressed the circumstances under whi...,None
1,104 S. Ct. 774,Can a flea market owner be held liable for cop...,A manufacturer or operator cannot be held liab...,"In this context, the citation from Sony Corp. ..."
2,104 S. Ct. 774,What constitutes contributory copyright infrin...,A technology provider is not liable for copyri...,None
3,239 F.3d 1004,Does the Digital Millennium Copyright Act (DMC...,The Digital Millennium Copyright Act (DMCA) pr...,None
4,239 F.3d 1004,Copyright infringement by downloading and dist...,"In the context of copyright law, downloading f...",None


In [138]:
Markdown(df_res[~df_res["application"].isna()].to_markdown(index=False))

| citation             | question_summary                                                                                                                                                                     | rule                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      | application                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
|:---------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 104 S. Ct. 774       | Can a flea market owner be held liable for copyright infringement by vendors?                                                                                                        | A manufacturer or operator cannot be held liable for copyright infringement merely because their product or service can be used to infringe copyrights, absent specific information that identifies infringing activity.                                                                                                                                                                                                                                                                                                                                                                                                  | In this context, the citation from Sony Corp. of America v. Universal City Studios, Inc. supports the principle that a flea market owner (like the video cassette recorder manufacturer in Sony) cannot be held liable for contributory copyright infringement simply because the structure of the flea market allows for the possibility that a vendor might sell infringing goods. However, this does not eliminate the possibility of liability if the flea market owner has constructive knowledge of the infringement based on specific facts.                                                                                                                                               |
| 239 F.3d 1004        | Columbus Farmers Market's defense against copyright infringement claims                                                                                                              | Third party direct liability as an element of secondary liability                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         | In the context of Columbus Farmers Market's argument, this legal ruling is relevant because it establishes that every case of contributory or vicarious liability necessarily involves the defendant being held accountable for the direct conduct of another. This ruling highlights the legal foundation for holding defendants responsible for the infringement actions of third parties, refuting the defense that damages were solely the result of third party actions over which the defendant had no knowledge, authority, or control.                                                                                                                                                    |
| 259 F. Supp. 2d 1029 | Does the sale of software that can be used for infringing activities constitute contributory copyright infringement?                                                                 | The sale of a product that is capable of substantial noninfringing uses does not automatically result in contributory infringement, even if the product might be used for illicit purposes. The court distinguished this principle from sole reliance on the product's potential for lawful use, suggesting that liability for contributory infringement requires actual knowledge of specific acts of infringement when the infringer is in a position to prevent those acts.                                                                                                                                            | In the case of Metro-Goldwyn-Mayer Studios, Inc. v. Grokster, Ltd., the district court concluded that as long as the software could be used for substantial noninfringing purposes, and there was no evidence of the defendants' timely knowledge of specific acts of infringement that they could prevent, they could not be found liable for contributory copyright infringement. This decision was based on the principle that the potential for lawful use of a product does not suffice to establish contributory infringement liability without actual knowledge of specific acts of infringement being demonstrated.                                                                       |
| 259 F. Supp. 2d 1029 | Does the law permit copyright infringing defendants to seek contribution from third parties?                                                                                         | Copyright infringing defendants cannot assert contribution against third parties for copyright infringement. Neither the Copyright Act nor federal common law recognize a copyright in-fringer’s right to contribution. Third-party direct liability is an element of an allegation of secondary liability, which involves either contributory or vicarious liability. Defendants can be held responsible for the infringement of third parties if they had knowledge of and materially contributed to that party’s infringing activity or had the ability to supervise and control it and financially benefited from it. | In the case of Metro-Goldwyn-Mayer Studios, Inc. v. Grokster, Ltd., it was clarified that every case of contributory or vicarious liability necessarily involves the defendant being held to answer for the direct conduct of another. The court elaborated on the aspects of contributory and vicarious liability, indicating that defendants cannot seek contribution from third parties for acts of copyright infringement. The defendants in this context are held liable for the actions of third parties (i.e., vendors) because they either had knowledge and materially contributed to the infringing activity or had the ability to supervise, control, and financially benefit from it. |
| 259 F. Supp. 2d 1029 | Does the decision to not hold Grokster contributorily liable for copyright infringement by users of its P2P file sharing program align with Verizon's interpretation of the statute? | The court held Grokster not contributorily liable for copyright infringement by users of its P2P file sharing program.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | Because the court agreed with Verizon's interpretation of the statute, which presumably leaned towards not holding service providers or platforms like Grokster contributorily liable under the circumstances described, the court reversed the district court's orders enforcing the subpoenas. This decision aligns with Verizon's stance and supports a broader interpretation that service platforms cannot be held contributorily liable for the actions of their users in such cases of copyright infringement, pending specific conditions and interpretations of the law.                                                                                                                 |
| 259 F.Supp.2d 1029   | Does actual knowledge of specific instances of infringement satisfy the 'knowledge' prong of contributory liability?                                                                 | The question of whether actual knowledge of specific instances of infringement is required for contributory liability focuses on the timing of the defendants' knowledge rather than the extent of knowledge possessed. For liability to attach, the defendant must have knowledge of direct infringement at the time they materially contribute to that infringement.                                                                                                                                                                                                                                                    | Despite Sinnott's reliance on Grokster to argue that actual knowledge of specific instances of infringement is required, the Grokster decision does not affirm this requirement for Sinnott's case. Instead, it emphasizes the timing of the knowledge in relation to the contribution to infringement, indicating that knowledge at the time of contribution is critical for establishing secondary liability.                                                                                                                                                                                                                                                                                   |
| 334 F.3d 643         | Inducement liability and DMCA safe harbors compatibility                                                                                                                             | The DMCA does not exempt from protection any mode of copyright liability, including under the doctrine of inducement. The requirements applied for all DMCA safe harbors do not inherently conflict with inducement liability.                                                                                                                                                                                                                                                                                                                                                                                            | A service provider could potentially be liable for inducement but still qualify for protection under DMCA safe harbors if they enforce a policy of removing repeat infringers. This suggests that inducement liability and DMCA safe harbor requirements can coexist without inherent contradiction.                                                                                                                                                                                                                                                                                                                                                                                              |
| 334 F.3d 643         | Defendants' liability regarding third party direct infringers                                                                                                                        | Defendants are legally responsible for the infringement of third parties when they have knowledge of and materially contribute to that third party’s infringing activity (contributory infringement), or when they have the ability to supervise and control it and financially benefit from it (vicarious liability).                                                                                                                                                                                                                                                                                                    | The court in the case cited reinforces that plaintiffs need not sue the numerous third party direct infringers in order to bring a cause of action against defendants. Additionally, the defendants may not seek contribution from those third party direct infringers.                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 464 U.S. 417         | Does the citation support the issuance of a § 512(h) subpoena to an ISP acting as a conduit for P2P file sharing?                                                                    | The court referenced Sony Corp. v. Universal City Studios, Inc., to emphasize that it's not the courts' role to rewrite the DMCA for unforeseen technologies, but rather Congress's role to address such issues.                                                                                                                                                                                                                                                                                                                                                                                                          | In the context given, the court deferred to the legislative branch for addressing the complexities brought about by new technologies like P2P file sharing, asserting that it would be inappropriate for the court to interpret or amend the DMCA in response to these technological advancements. This directly impacts the question of the legality of issuing § 512(h) subpoenas to ISPs in P2P contexts, indicating that such a determination is for Congress to make, not the courts.                                                                                                                                                                                                        |

In [71]:
all_data[0].model_dump()

{'citation': '104 S. Ct. 774',
 'question_summary': 'Understanding the legal principle established by Sony Corp. of America v. Universal City Studios, Inc.',
 'rule': "The case addressed the circumstances under which secondary liability for copyright infringement could arise from the distribution of a product capable of copying copyrighted material, specifically considering a product distributor's intent and encouragement of the product's use for infringement.",
 'application': None}

In [121]:
def save_models_to_json(models: List[BaseModel], file_path: str) -> None:
    """
    Saves a list of Pydantic models to a JSON file.

    Args:
        models: A list of Pydantic model instances.
        file_path: The path to the JSON file where the data will be saved.

    Returns:
        None
    """
    with open(file_path, 'w') as f:
        # Convert list of models to list of dictionaries
        data = [model.model_dump() for model in models]
        json.dump(data, f, indent=4)

In [122]:
save_models_to_json(all_data, 'models.json')

In [124]:
import json
from typing import List, Type, TypeVar
from pydantic import BaseModel

T = TypeVar('T', bound=BaseModel)

def load_models_from_json(model_class: Type[T], file_path: str) -> List[T]:
    """
    Loads JSON data from a file and converts it into a list of Pydantic models.

    Args:
        model_class: The Pydantic model class to which the JSON objects will be converted.
        file_path: The path to the JSON file from which the data will be loaded.

    Returns:
        A list of Pydantic model instances.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
        models = [model_class.model_validate(item) for item in data]
    return models

In [133]:
from pprint import pprint

models = load_models_from_json(LegalCitation, 'models.json')
pprint(models[0].model_dump())

{'application': None,
 'citation': '104 S. Ct. 774',
 'question_summary': 'Understanding the legal principle established by Sony '
                     'Corp. of America v. Universal City Studios, Inc.',
 'rule': 'The case addressed the circumstances under which secondary liability '
         'for copyright infringement could arise from the distribution of a '
         'product capable of copying copyrighted material, specifically '
         "considering a product distributor's intent and encouragement of the "
         "product's use for infringement."}


In [157]:
Markdown(df_res[["citation", "question_summary", "rule"]].to_markdown(index=False))

| citation             | question_summary                                                                                                                                                                                                                      | rule                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
|:---------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| 104 S. Ct. 774       | Understanding the legal principle established by Sony Corp. of America v. Universal City Studios, Inc.                                                                                                                                | The case addressed the circumstances under which secondary liability for copyright infringement could arise from the distribution of a product capable of copying copyrighted material, specifically considering a product distributor's intent and encouragement of the product's use for infringement.                                                                                                                                                                                                                                                                                                                  |
| 104 S. Ct. 774       | Can a flea market owner be held liable for copyright infringement by vendors?                                                                                                                                                         | A manufacturer or operator cannot be held liable for copyright infringement merely because their product or service can be used to infringe copyrights, absent specific information that identifies infringing activity.                                                                                                                                                                                                                                                                                                                                                                                                  |
| 104 S. Ct. 774       | What constitutes contributory copyright infringement?                                                                                                                                                                                 | A technology provider is not liable for copyright infringement committed by end users if the product sold is widely used for legitimate, unobjectionable purposes, or is merely capable of substantial noninfringing uses.                                                                                                                                                                                                                                                                                                                                                                                                |
| 239 F.3d 1004        | Does the Digital Millennium Copyright Act (DMCA) provide safe harbor protection to individuals or entities that have been found liable for inducing copyright infringement?                                                           | The Digital Millennium Copyright Act (DMCA) provides safe harbor protection to individuals or entities even if they have been found liable for inducing copyright infringement, under certain circumstances.                                                                                                                                                                                                                                                                                                                                                                                                              |
| 239 F.3d 1004        | Copyright infringement by downloading and distributing copyright-protected materials                                                                                                                                                  | In the context of copyright law, downloading files containing copyrighted material without authorization violates the copyright owner's reproduction rights, and making those files available for others to download infringes upon the owner's exclusive right to distribute the material.                                                                                                                                                                                                                                                                                                                               |
| 239 F.3d 1004        | Does Section 512(h) of the Digital Millennium Copyright Act authorize the issuance of a subpoena to an Internet Service Provider acting solely as a conduit for communications to identify users suspected of copyright infringement? | The court ruled in favor of Verizon's interpretation of the statute, therefore reversing the district court's enforcement of the subpoenas without addressing the constitutional arguments.                                                                                                                                                                                                                                                                                                                                                                                                                               |
| 239 F.3d 1004        | Do users who download files containing copyrighted music via peer-to-peer networks infringe on the copyright holder’s reproduction rights?                                                                                            | Napster users who download files containing copyrighted music violate plaintiffs’ reproduction rights.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 239 F.3d 1004        | Does inducement to infringe copyrights lead to irreparable harm and undermine the control of intellectual property?                                                                                                                   | The court rejected the request for compulsory royalties as opposed to injunctive relief because Plaintiffs would lose the power to control their intellectual property. The inducement of infringement greatly erodes the ability of copyright owners to enforce their exclusive rights and does not offer a realistic mechanism for collecting statutory damages for all subsequent infringements occurring outside of the system and software in use.                                                                                                                                                                   |
| 239 F.3d 1004        | What is the significance of the ruling in A&M Records, Inc. v. Napster, Inc., with regard to contributory copyright infringement and the requirement for knowledge of specific infringing files?                                      | Evidence of actual knowledge of specific acts of infringement is required to hold a computer system operator liable for contributory copyright infringement.                                                                                                                                                                                                                                                                                                                                                                                                                                                              |
| 239 F.3d 1004        | Is building a file-sharing network to foster a user base for another service, considering the network enables copyrighted material to be shared, legally permissible?                                                                 | Analysis of contributory copyright infringement with respect to the operation of a file-sharing network.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  |
| 239 F.3d 1004        | Columbus Farmers Market's defense against copyright infringement claims                                                                                                                                                               | Third party direct liability as an element of secondary liability                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| 239 F.3d 1004        | Does 17 U.S.C. § 512(h) authorize the issuance of a subpoena to an ISP acting solely as a conduit for communications to identify alleged copyright infringers without an underlying case or controversy?                              | Section 512(h) does not authorize the issuance of a subpoena to an ISP acting solely as a conduit for communications the content of which is determined by others. The decision of the district court enforcing the subpoenas was reversed.                                                                                                                                                                                                                                                                                                                                                                               |
| 239 F.3d 1004        | Explanation of the Napster system                                                                                                                                                                                                     | Provides a detailed description of the Napster peer-to-peer file-sharing system                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 239 F.3d 1004        | Do MFM vendors directly infringe copyright by selling unauthorized copies of copyrighted sound recordings?                                                                                                                            | Establishing direct copyright infringement by vendors is a prerequisite to both the contributory and vicarious copyright infringement claims.                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| 239 F.3d 1004        | Can downloading and uploading copyrighted music through a peer-to-peer service constitute direct infringement?                                                                                                                        | Downloading and uploading copyrighted material are infringements of copyright law.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        |
| 259 F. Supp. 2d 1029 | Does the sale of software that can be used for infringing activities constitute contributory copyright infringement?                                                                                                                  | The sale of a product that is capable of substantial noninfringing uses does not automatically result in contributory infringement, even if the product might be used for illicit purposes. The court distinguished this principle from sole reliance on the product's potential for lawful use, suggesting that liability for contributory infringement requires actual knowledge of specific acts of infringement when the infringer is in a position to prevent those acts.                                                                                                                                            |
| 259 F. Supp. 2d 1029 | What constitutes copyright infringement with regard to downloading copyrighted media files over the internet?                                                                                                                         | Downloading copyrighted media files infringes the copyright holder's rights of reproduction and constitutes direct copyright infringement.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 259 F. Supp. 2d 1029 | What was the legal principle established in Metro-Goldwyn-Mayer Studios, Inc. v. Grokster, Ltd.?                                                                                                                                      | The court held that Grokster was not contributorily liable for copyright infringement by users of its P2P file sharing program.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 259 F. Supp. 2d 1029 | Does downloading copyrighted music without authorization constitute an unlawful reproduction of the work?                                                                                                                             | Downloading copyrighted sound recordings without the authorization of the copyright holder constitutes an unlawful reproduction of the work in violation of § 106(1) of the Copyright Act.                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 259 F. Supp. 2d 1029 | Does refusal to provide certain information and allowing other networks to offer unfiltered software constitute waiver of the right to a permanent injunction?                                                                        | Waiver is the intentional relinquishment of a known right with knowledge of its existence and the intent to relinquish it. Waiver of copyright occurs only with the intent to surrender rights.                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 259 F. Supp. 2d 1029 | What was the legal issue in the case where the court granted summary judgment for Defendants StreamCast and Grokster?                                                                                                                 | The legal issue revolved around whether Defendants’ file-sharing software contributed to massive copyright infringement of copyrighted works owned by Plaintiffs.                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| 259 F. Supp. 2d 1029 | Does the law permit copyright infringing defendants to seek contribution from third parties?                                                                                                                                          | Copyright infringing defendants cannot assert contribution against third parties for copyright infringement. Neither the Copyright Act nor federal common law recognize a copyright in-fringer’s right to contribution. Third-party direct liability is an element of an allegation of secondary liability, which involves either contributory or vicarious liability. Defendants can be held responsible for the infringement of third parties if they had knowledge of and materially contributed to that party’s infringing activity or had the ability to supervise and control it and financially benefited from it. |
| 259 F. Supp. 2d 1029 | Does the decision to not hold Grokster contributorily liable for copyright infringement by users of its P2P file sharing program align with Verizon's interpretation of the statute?                                                  | The court held Grokster not contributorily liable for copyright infringement by users of its P2P file sharing program.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |
| 259 F. Supp. 2d 1029 | Is file-sharing software distribution that facilitates copyright material exchange subject to vicarious and contributory copyright infringement?                                                                                      | Metro-Goldwyn-Mayer Studios, Inc. v. Grokster, Ltd. determined that distributors of peer-to-peer file-sharing software might be liable for copyright infringement if the software is predominantly used to exchange copyrighted material, pursuant to 17 U.S.C. §§ 501-13.                                                                                                                                                                                                                                                                                                                                                |
| 259 F.Supp.2d 1029   | Does actual knowledge of specific instances of infringement satisfy the 'knowledge' prong of contributory liability?                                                                                                                  | The question of whether actual knowledge of specific instances of infringement is required for contributory liability focuses on the timing of the defendants' knowledge rather than the extent of knowledge possessed. For liability to attach, the defendant must have knowledge of direct infringement at the time they materially contribute to that infringement.                                                                                                                                                                                                                                                    |
| 259 F. Supp. 2d 1029 | What does the ruling in Metro-Goldwyn-Mayer Studios, Inc. v. Grokster, Ltd. indicate about liability for file sharing?                                                                                                                | The district court in Grokster ruled that if either defendant ceased operations and deactivated all computers they control, users of their products would still be able to continue sharing files with little or no interruption.                                                                                                                                                                                                                                                                                                                                                                                         |
| 334 F.3d 643         | Inducement liability and DMCA safe harbors compatibility                                                                                                                                                                              | The DMCA does not exempt from protection any mode of copyright liability, including under the doctrine of inducement. The requirements applied for all DMCA safe harbors do not inherently conflict with inducement liability.                                                                                                                                                                                                                                                                                                                                                                                            |
| 334 F.3d 643         | Can imposing liability on software distributors for users' infringement limit technological innovation?                                                                                                                               | Secondary liability for contributory or vicarious infringement                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
| 334 F.3d 643         | Defendants' liability regarding third party direct infringers                                                                                                                                                                         | Defendants are legally responsible for the infringement of third parties when they have knowledge of and materially contribute to that third party’s infringing activity (contributory infringement), or when they have the ability to supervise and control it and financially benefit from it (vicarious liability).                                                                                                                                                                                                                                                                                                    |
| 334 F.3d 643         | How should the standard of 'substantial noninfringing use' be interpreted?                                                                                                                                                            | The Seventh Circuit interpreted the 'substantial noninfringing use' standard to include the consideration of how 'probable' the noninfringing uses of a product are.                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| 334 F.3d 643         | What is the concept of willful blindness in copyright law and its implications for contributory liability?                                                                                                                            | Willful blindness is knowledge, in copyright law ... as it is in the law generally.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       |
| 464 U.S. 417         | How did the Supreme Court address the issue of secondary copyright infringement in the context of new technologies?                                                                                                                   | Sony considered whether secondary liability for infringement could arise solely from the distribution of a commercial product capable of copying copyrighted material, focusing on the Betamax video tape recorder made by Sony.                                                                                                                                                                                                                                                                                                                                                                                          |
| 464 U.S. 417         | Does the citation support the issuance of a § 512(h) subpoena to an ISP acting as a conduit for P2P file sharing?                                                                                                                     | The court referenced Sony Corp. v. Universal City Studios, Inc., to emphasize that it's not the courts' role to rewrite the DMCA for unforeseen technologies, but rather Congress's role to address such issues.                                                                                                                                                                                                                                                                                                                                                                                                          |
| 464 U.S. 417         | Are the terms "knowingly materially contributing", "enabling", and "permitting" in the injunction against StreamCast relevant or necessary given its applicability to different infringement theories?                                | Sony Corp. v. Universal City Studios, Inc. introduces a standard for evaluating technology capable of substantial noninfringing uses.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     |
| 464 U.S. 417         | How does the concept of contributory liability for copyright infringement based on distribution of a product apply, according to Sony Corp. of America v. Universal City Studios, Inc.?                                               | Distribution of a commercial product capable of substantial non-infringing uses cannot give rise to contributory liability for infringement unless the distributor had actual knowledge of specific instances of infringement and failed to act on that knowledge.                                                                                                                                                                                                                                                                                                                                                        |
| 464 U.S. 417         | Does the legislative history support the issuance of a § 512(h) subpoena to an ISP acting as a conduit for P2P file sharing?                                                                                                          | The courts cannot rewrite the DMCA to fit new and unforeseen internet architecture; addressing the plight of copyright holders is the responsibility of Congress.                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| 464 U.S. 417         | Can the sale of video tape recorders give rise to contributory copyright infringement liability?                                                                                                                                      | No, the sale of video tape recorders cannot give rise to contributory copyright infringement liability if the product is capable of substantial or commercially significant noninfringing uses.                                                                                                                                                                                                                                                                                                                                                                                                                           |
| 464 U.S. 417         | How is secondary liability for copyright infringement established?                                                                                                                                                                    | Courts have long recognized that in certain circumstances, vicarious or contributory liability for copyright infringement will be imposed.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                |
| 464 U.S. 417         | Cox's liability for contributory infringement                                                                                                                                                                                         | the Supreme Court’s decision established that an internet service capable of substantial noninfringing uses should generally be immunized from liability for contributory infringement                                                                                                                                                                                                                                                                                                                                                                                                                                    |